In [1]:
import jupyter_aws_sso
import time
import importlib
importlib.reload(jupyter_aws_sso)

jupyter_aws_sso.login("Jupyter-IR-AdministratorAccess")


Profile found
If the windows doesn't automatically open, click on this https://device.sso.us-east-1.amazonaws.com/?user_code=XZXR-CDTD to activate the session


<IPython.core.display.Javascript object>

# Organization Configuration
This notebook will configure the organization to have the resources needed to run the Jupyter Incident Response runbooks across the organization. This **must be run in the organizational managemebnt account.**.

The Athena tables can be set up in a delagate account so no resources will be deployed into the management account.

## Accounts:
* **Management Account**: The root account that owns the AWS organization. Typically resources are note deployed to the root account, but some configurations must be done in this account. For example, the cloudtrail organizational trail is created in the management account.
* **Logging**: The account that holds the log files. Typically this account contains the buckets for the CloudTrail and other logs, and Athena tables used to query them.
* **CloudFormation StackSet Delegated Account**: An account designated by the management account to own stacksets.

## Parameters
The following cell holds the parameters for the environment

* **athena_database**: The name of the athena database that will be used to query logs in S3.
* **athena_cloudtrail_table**: The name of the athena table that will be used to query CloutTrail logs in S3.
* **flow_logs_bucket**: The bucket that holds the organizational flow logs. Leave empty and one will be created automatically.
* **enable_flow_logs_automatically**: Automatically enable flow logs in all VPCs in the organization. See [blog](https://aws.amazon.com/blogs/mt/how-to-enable-vpc-flow-logs-automatically-using-aws-config-rules/)


In [73]:
# Save Settings


external_cloud_trail = False # Set this to True if the organization cloudtrail is managed externally.
athena_database = "jupyter-incident-response"  # <--- To customize, replace the value with the athena database name
athena_cloudtrail_table = "cloudtrail_logs"  # <--- To customize, replace the value with the athena table name for cloudtrail logs
flow_logs_bucket = ""  # <--- Leave blank and a bucket will be created automatically
flowlog_output_format = "parquet"
cloudtrail_bucket = ""  # <--- Leave blank and a bucket will be created automatically
enable_flow_logs_automatically = True
opt_in_out = "in"
region_list = "us-east-1,us-west-2,us-east-2"
aggregator_id = ""  # <--- Leave blank and an aggregator will be created automatically


## CloudFormation
This notebook uses cloudformation to deploy the cloudformation stacks and stack sets used to configure the organization. 

The templates have feature toggles to turn on an off the creation of the configuration. If the resources have already been created outside of this notebook, the resource creation will be skipped.
* **logging-account.yaml**: 
  * If necessary. creates the S3 buckets for organization-wide logging
  * Creates Athena tables to query logs
* **account-level-resources.yaml**:
  * Note: Deployed as a stackset from the **Delegated Stack Stack** account to every account under the root.
  * Creates global resources in each member account.
  * Creates a stackset to deployed to each region in the account
    * Configures Flow Log Auto Remediation
    * 
* **managage-account.yaml**: If necessary, configures organizational CloudTrail logs.

# Organizational Onboarding

The notebook will validate and update the organization to configure the logging and configuration of the organization to the other runbooks have access to the data and services needed to execute the runbooks.

This runbook is designed to be able to be run multiple times, so it will not create additional resources if the configurate has already been found.

The following services will be configured.
* **CloudTrail**: An organization trail will be configured to store the CloudTrail logs in a bucket in the management account. The first trail in each account is included free of charge. If an organzation trail already exists, a new one will not be created.
* **VPC Flow Logs**: Any VPCs with the tag 'runbooks-enable-flowlogs' set to 'true' will have VPC flow logs enabled into a central bucket.
* **Athena**: Athena will be configured to query the following service logs:
  * CloudTrail
  * VPC Flow Logs
  * **Up Next**: AWS Config
  * ***Future*** Application Load Balancer
  * ***Future*** Elastic Load Balancer
  * ***Future*** CloudFront
  * ***Future*** Amazon EMR
  * ***Future*** Global Accelerator
  * ***Future*** GuardDuty
  * ***Future*** Network Firewall
  * ***Future*** Network Load Balancer
  * ***Future*** Route 53
  * ***Future*** Amazon SES
  * ***Future*** AWS WAF
  

## Load Settings

The information on how to connect to data in the AWS environment is managed through SSM Parameter Store.

The athena database and tablenames can be provided below. The runbook will manage the Athena tables so they are configured with the columns and partitioning expected by the notebooks.

In [3]:
# Load Settings
import boto3
ssm_client = boto3.client('ssm')

put_response = ssm_client.put_parameter(Name='Jupyter-Athena-Database', Value=athena_database, Type="String", Overwrite=True)
sso_portal_url_response = ssm_client.put_parameter(Name='Jupyter-Athena-Cloudtrail', Value=athena_cloudtrail_table, Type="String", Overwrite=True)

get_management_account_response = ssm_client.get_parameter(Name='Jupyter-Management-Account')
get_cfn_delegate_account_response = ssm_client.get_parameter(Name='Jupyter-CfnDelegateAccount')
get_logging_account_response = ssm_client.get_parameter(Name='Jupyter-LoggingAccount')

management_account = get_management_account_response['Parameter']['Value']
cfn_delegate_account = get_cfn_delegate_account_response['Parameter']['Value']
logging_account = get_logging_account_response['Parameter']['Value']

ec2_client = boto3.client('ec2')

stack_set_region_list = ""

if opt_in_out == "out":
    region_opt_out = region_list.split(',')
    stack_set_regions = []

    region_response = ec2_client.describe_regions()

    for region in region_response['Regions']:
        if not region['RegionName'] in region_opt_out:
            stack_set_regions.append(region['RegionName'])
    stack_set_region_list =','.join(stack_set_regions)
else:
    stack_set_region_list = region_list

management_account

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


'383086473915'

In [4]:
# Configure Logging Account
org_client = boto3.client('organizations')
desc_client = org_client.describe_organization()

org_id = desc_client['Organization']['Id']

profile = f"Jupyter-IR-AdministratorAccess-{logging_account}"

!aws cloudformation deploy --template-file cfn-templates/logging-account.yaml --stack-name organization-logging --parameter-overrides RegionList=$stack_set_region_list FlowLogsFormat=$flowlog_output_format DatabaseName=$athena_database CloudTrailLogsBucket=$cloudtrail_bucket FlowLogsBucket=$flow_logs_bucket OrgId=$org_id CloudTrailTableName=$athena_cloudtrail_table --capabilities CAPABILITY_IAM --profile $profile


Waiting for changeset to be created..

No changes to deploy. Stack organization-logging is up to date


**Note** Organizational Rules don't support remediation, so the rule will be deployed in each region.


In [5]:
config_bucket = !aws cloudformation describe-stacks --stack-name organization-logging  --profile $profile --query "Stacks[0].Outputs[?OutputKey=='ConfigBucket'].OutputValue" --output text
config_bucket = config_bucket[0]

config_topic = !aws cloudformation describe-stacks --stack-name organization-logging  --profile $profile --query "Stacks[0].Outputs[?OutputKey=='ConfigTopic'].OutputValue" --output text
config_topic = config_topic[0]


cfn_bucket = !aws cloudformation describe-stacks --stack-name organization-logging  --profile $profile --query "Stacks[0].Outputs[?OutputKey=='CfnBucket'].OutputValue" --output text
cfn_bucket = cfn_bucket[0]


cfn_bucket = !aws cloudformation describe-stacks --stack-name organization-logging  --profile $profile --query "Stacks[0].Outputs[?OutputKey=='CfnBucket'].OutputValue" --output text
cfn_bucket = cfn_bucket[0]

cfn_cloudtrail_bucket = !aws cloudformation describe-stacks --stack-name organization-logging --profile $profile  --query "Stacks[0].Outputs[?OutputKey=='CloudTrailLogsBucket'].OutputValue" --output text
cfn_flowlog_bucket = !aws cloudformation describe-stacks --stack-name organization-logging --profile $profile  --query "Stacks[0].Outputs[?OutputKey=='BucketName'].OutputValue" --output text

cfn_cloudtrail_bucket = cfn_cloudtrail_bucket[0]
cfn_flowlog_bucket = cfn_flowlog_bucket[0]

print(f"config_bucket={config_bucket}\nconfig_topic={config_topic}\ncfn_bucket={cfn_bucket}\ncfn_cloudtrail_bucket={cfn_cloudtrail_bucket}\ncfn_flowlog_bucket={cfn_flowlog_bucket}")


config_bucket=organization-logging-configbucket-szp9h6u3hmx3
config_topic=arn:aws:sns:us-east-1:251344881676:config-topic-251344881676-organization-logging
cfn_bucket=organization-logging-cfnbucket-11jngwyngudcw
cfn_cloudtrail_bucket=organization-logging-cfncloudtraillogsbucket-l5ofpp3k3bdm
cfn_flowlog_bucket=organization-logging-flowlogbucket-2c7atewedw2g


## Validate CloudTrail
The next section will check the CloudTrail logs and validate an organizational trail is configured.

Then the cloudformation stack will be created in the management account to configure the organization trail if one is needed.

In [6]:
flowlog_output_format

'parquet'

In [7]:


org_trail_info = {}
org_trail_arn = ""
found_org_trail = True

cloudtrail_client = boto3.client('cloudtrail', region_name = "us-east-1")

if external_cloud_trail == True:
    found_org_trail = False

    paginator = cloudtrail_client.get_paginator('list_trails')

    response_iterator = paginator.paginate()
    for page in response_iterator:
        for trail in page["Trails"]:
            trail_response = cloudtrail_client.get_trail(Name=trail['TrailARN'])
            if trail_response['Trail']['IsOrganizationTrail'] and trail_response['Trail']['IsMultiRegionTrail']:
                found_org_trail = True
                org_trail_info = trail_response['Trail']
                org_trail_arn = org_trail_info['TrailARN']

    if not found_org_trail:
        raise ("External cloud trail not found, please configure.")

!aws cloudformation deploy --template-file cfn-templates/managage-account.yaml --stack-name management-organization --parameter-overrides LoggingAccount=$logging_account CloudTrailArn=$org_trail_arn TrailBucket=$cfn_cloudtrail_bucket --capabilities CAPABILITY_IAM



Waiting for changeset to be created..

No changes to deploy. Stack management-organization is up to date


In [8]:
trail_arn = !aws cloudformation describe-stacks --stack-name management-organization --query "Stacks[0].Outputs[?OutputKey=='TrailArn'].OutputValue" --output text

trail_response = cloudtrail_client.get_trail(Name=trail_arn[0])
org_trail_info = trail_response['Trail']

print(f"Organization Trail was found: {org_trail_info['Name']} in region {org_trail_info['HomeRegion']}")

Organization Trail was found: trail-management-organization in region us-east-1


### Create Trail, if needed
If an organization trail was not found, the following section will create one using a cloudformation template.

## Account and Region Configurations

Any configurations that need to deployed to each account will be handled with this stackset.


# Configure member accounts for flow logs and AWS Config
The next section will create a CloudFormation stack that configures the AWS Config and flowlogs in every account and every region.



In [9]:

org_unit_id = org_client.list_roots()['Roots'][0]['Id']


In [10]:
import random, string

config_bucket = '' # Disable config recorder 

profile = f"Jupyter-IR-AdministratorAccess-{logging_account}"
random_str=''.join(random.choices(string.ascii_uppercase + string.digits, k=8))

!aws s3 sync cfn-templates/ s3://$cfn_bucket/$random_str/ --profile $profile > /dev/null

profile = f"Jupyter-IR-AdministratorAccess-{cfn_delegate_account}"
template_url = f'https://s3.us-east-1.amazonaws.com/{cfn_bucket}/{random_str}'

!aws cloudformation deploy --stack-name jupyter-ir-config --template-file cfn-templates/organization-ir-config.yaml --parameter-overrides ConfigTopic=$config_topic ConfigBucket=$config_bucket OrgUnitId=$org_unit_id OutputFormat=$flowlog_output_format  AutomateRemediationTemplateUrl=$template_url RegionList=$stack_set_region_list --profile $profile --capabilities CAPABILITY_NAMED_IAM




Waiting for changeset to be created..
Waiting for stack create/update to complete
Successfully created/updated stack - jupyter-ir-config


In [11]:

profile = f"Jupyter-IR-AdministratorAccess-{logging_account}"
!aws s3 rm --recursive s3://$cfn_bucket/$random_str --profile $profile > /dev/null

## Test CloudTrail access with Athena
The following code uses a helper library to query cloudtrail logs and return them as a dataframe

In [12]:
import jupyter_athena
import importlib
importlib.reload(jupyter_athena)

accountid = "383086473915"

sql = f"""
SELECT * 
FROM \"{athena_database}\".\"{athena_cloudtrail_table}\"
WHERE accountId = '{accountid}'
LIMIT 10
"""

display(jupyter_athena.run_query(sql))


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


profile_name=Jupyter-IR-ViewOnly-251344881676


,eventversion,useridentity,eventtime,eventsource,eventname,awsregion,sourceipaddress,useragent,errorcode,errormessage,...,eventtype,apiversion,readonly,recipientaccountid,serviceeventdetails,sharedeventid,vpcendpointid,accountid,region,timestamp
0,1.08,"{type=AssumedRole, principalid=AROAVSMN3WK5UHW...",2022-10-20T18:58:36Z,s3.amazonaws.com,PutObject,us-west-2,AWS Internal,AWS Internal,AccessDenied,Access Denied,...,AwsApiCall,None,false,383086473915,None,ddbccacf-5762-40e0-8a15-2adc03b09024,None,383086473915,us-west-2,2022/10/20
1,1.08,"{type=AssumedRole, principalid=AROAVSMN3WK5UHW...",2022-10-20T18:59:43Z,s3.amazonaws.com,PutObject,us-west-2,AWS Internal,AWS Internal,AccessDenied,Access Denied,...,AwsApiCall,None,false,383086473915,None,19513e1e-3439-4219-b592-bd7754663536,None,383086473915,us-west-2,2022/10/20
2,1.08,"{type=AWSService, principalid=null, arn=null, ...",2022-10-20T04:53:29Z,sts.amazonaws.com,AssumeRole,us-west-2,config.amazonaws.com,config.amazonaws.com,None,None,...,AwsApiCall,None,true,383086473915,None,03abfa45-52f0-4f2a-9fa1-4eb11c93bcb9,None,383086473915,us-west-2,2022/10/20
3,1.08,"{type=AWSService, principalid=null, arn=null, ...",2022-10-20T12:50:42Z,sts.amazonaws.com,AssumeRole,us-west-2,securityhub.amazonaws.com,securityhub.amazonaws.com,None,None,...,AwsApiCall,None,true,383086473915,None,ee2a2dce-3a20-47d1-a4f8-8eef2de31c76,None,383086473915,us-west-2,2022/10/20
4,1.08,"{type=AssumedRole, principalid=AROAIV7SQSUTYYW...",2022-10-20T12:50:46Z,config.amazonaws.com,DescribeConfigRules,us-west-2,securityhub.amazonaws.com,securityhub.amazonaws.com,None,None,...,AwsApiCall,None,true,383086473915,None,None,None,383086473915,us-west-2,2022/10/20
5,1.08,"{type=AssumedRole, principalid=AROAIV7SQSUTYYW...",2022-10-20T12:50:50Z,config.amazonaws.com,DescribeConfigRules,us-west-2,securityhub.amazonaws.com,securityhub.amazonaws.com,None,None,...,AwsApiCall,None,true,383086473915,None,None,None,383086473915,us-west-2,2022/10/20
6,1.08,"{type=AssumedRole, principalid=AROAIV7SQSUTYYW...",2022-10-20T12:50:43Z,config.amazonaws.com,DescribeConfigRules,us-west-2,securityhub.amazonaws.com,securityhub.amazonaws.com,None,None,...,AwsApiCall,None,true,383086473915,None,None,None,383086473915,us-west-2,2022/10/20
7,1.08,"{type=AssumedRole, principalid=AROAIV7SQSUTYYW...",2022-10-20T12:50:47Z,config.amazonaws.com,DescribeConfigRules,us-west-2,securityhub.amazonaws.com,securityhub.amazonaws.com,None,None,...,AwsApiCall,None,true,383086473915,None,None,None,383086473915,us-west-2,2022/10/20
8,1.08,"{type=AssumedRole, principalid=AROAIV7SQSUTYYW...",2022-10-20T12:50:49Z,config.amazonaws.com,DescribeConfigRules,us-west-2,securityhub.amazonaws.com,securityhub.amazonaws.com,None,None,...,AwsApiCall,None,true,383086473915,None,None,None,383086473915,us-west-2,2022/10/20
9,1.08,"{type=AssumedRole, principalid=AROAIV7SQSUTYYW...",2022-10-20T12:50:48Z,config.amazonaws.com,DescribeConfigRules,us-west-2,securityhub.amazonaws.com,securityhub.amazonaws.com,None,None,...,AwsApiCall,None,true,383086473915,None,None,None,383086473915,us-west-2,2022/10/20


## Flow Logs
Enable athena to query organizational flow logs.

This needs to be deployed in two steps. 
1. Deploy CFN The central bucket and athena query in the management account
1. The stacksets for all regions/accounts in organization.

In [13]:


display(jupyter_athena.get_vpc_flow_by_account("913149361159"))


/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


profile_name=Jupyter-IR-ViewOnly-251344881676


,interface_id,srcaddr,srcport,dstaddr,dstport,flow_count,packet_count,sum_bytes
0,eni-05ac490193a65bd0f,45.144.154.81,19201,10.0.0.189,23,340,535,21400
1,eni-05ac490193a65bd0f,87.227.99.86,27343,10.0.0.189,23,184,197,7880
2,eni-05ac490193a65bd0f,61.99.254.70,47593,10.0.0.189,23,155,188,7520
3,eni-05ac490193a65bd0f,99.6.136.160,62810,10.0.0.189,23,22,22,880
4,eni-05ac490193a65bd0f,223.10.63.57,13052,10.0.0.189,23,21,31,1240
...,...,...,...,...,...,...,...,...
95,eni-05ac490193a65bd0f,24.166.74.146,11017,10.0.0.189,23,2,2,88
96,eni-05ac490193a65bd0f,170.187.163.250,61000,10.0.0.189,25,2,2,80
97,eni-05ac490193a65bd0f,66.228.37.12,61000,10.0.0.189,995,2,2,80
98,eni-05ac490193a65bd0f,170.187.163.218,61000,10.0.0.189,563,2,2,80


In [14]:
# Delete Flow Logs
import boto3

# delete this in the future or move to another notebook

if False: # Don't do this by default, it exists to delete all the flow logs in an org if the deployment needs to be redone.

    accounts = ["913149361159","251344881676", "959309154643", "669399753706" ]



    for account in accounts:
        profile = f"Jupyter-IR-AdministratorAccess-{account}"
        !aws s3 rm --recursive s3://$cfn_flowlog_bucket/AWSLogs/$account --profile Jupyter-IR-AdministratorAccess-251344881676
        for region_name in stack_set_region_list.split(','):
            session = boto3.session.Session(profile_name=profile, region_name=region_name)
            client = session.client('ec2')
            response = client.describe_flow_logs()

            flow_log_ids = []
            for flow_log in response['FlowLogs']:
                flow_log_id = flow_log['FlowLogId']
                flow_log_ids.append(flow_log_id)

            if len(flow_log_ids) > 0:
                delete_response = client.delete_flow_logs(FlowLogIds=flow_log_ids)
                print(f"deleted: {flow_log_ids} in region {region_name} in account {account}")

In [15]:


for account in accounts:
    profile = f"Jupyter-IR-AdministratorAccess-{account}"
    for region_name in stack_set_region_list.split(','):
        session = boto3.session.Session(profile_name=profile, region_name=region_name)
        client = session.client('ec2')
        response = client.describe_flow_logs()

        flow_log_ids = []
        for flow_log in response['FlowLogs']:
            flow_log_id = flow_log['FlowLogId']
            print(f"FlowLogId: {flow_log_id}:{flow_log['DestinationOptions']['FileFormat']} in region {region_name} in account {account}")

NameError: name 'accounts' is not defined

## AWS Config

It is a best practice to enable AWS config across the organization to collect data on the current state of the resoruces deployed in the organization, establish detective controls for compliance, and make resources available through a query interface.

In [ ]:
# Get all the accounts in the organization
org_client = boto3.client('organizations')
paginator = org_client.get_paginator('list_accounts')

accounts = []

page_iterator = paginator.paginate()
for page in page_iterator:
    for account in page['Accounts']:
        accounts.append(account['Id'])
        
accounts

### Check delivery channels

Verify a delivery channel exists for the every account and every region im the organizational.


In [17]:

accounts = []

paginator = org_client.get_paginator('list_accounts')
page_iterator = paginator.paginate()
for page in page_iterator:        
    for acct in page['Accounts']:
        accounts.append(acct['Id'])

for account in accounts:
    profile = f"Jupyter-IR-AdministratorAccess-{account}"
    for region_name in stack_set_region_list.split(','):
        
        session = boto3.session.Session(profile_name=profile, region_name=region_name)
        config_client = session.client('config')
        delivery_channels_result = config_client.describe_delivery_channels()
        delivery_channel_found = False
        for delivery_channel in delivery_channels_result['DeliveryChannels']:
            delivery_channel_found = True
        
        if delivery_channel_found:
            print(f"Delivery Channel found in account {account} in region {region_name}")
        else:
            print(f"Delivery Channel NOT found in account {account} in region {region_name}")

Delivery Channel found in account 383086473915 in region us-east-1
Delivery Channel found in account 383086473915 in region us-west-2
Delivery Channel found in account 383086473915 in region us-east-2
Delivery Channel found in account 669399753706 in region us-east-1
Delivery Channel found in account 669399753706 in region us-west-2
Delivery Channel found in account 669399753706 in region us-east-2
Delivery Channel found in account 913149361159 in region us-east-1
Delivery Channel found in account 913149361159 in region us-west-2
Delivery Channel found in account 913149361159 in region us-east-2
Delivery Channel found in account 251344881676 in region us-east-1
Delivery Channel found in account 251344881676 in region us-west-2
Delivery Channel found in account 251344881676 in region us-east-2
Delivery Channel found in account 959309154643 in region us-east-1
Delivery Channel found in account 959309154643 in region us-west-2
Delivery Channel found in account 959309154643 in region us-ea

## Set up the multi-account multi-region aggregator

An aggregator is an AWS Config resource type that collects AWS Config configuration and compliance data from the following:

* Multiple accounts and multiple regions.
* Single account and multiple regions.
* An organization in AWS Organizations and all the accounts in that organization which have AWS Config enabled.
* Use an aggregator to view the resource configuration and compliance data recorded in AWS Config.

In [74]:
profile = f"Jupyter-IR-AdministratorAccess-{management_account}"
!aws cloudformation deploy --stack-name aws-config-aggregator --template-file cfn-templates/aws-config-aggregator.yaml --parameter-overrides AggregatorId=$aggregator_id RegionList=$stack_set_region_list --profile $profile --capabilities CAPABILITY_IAM

print('It can take 10 minutes for the aggregator to be ready to field a query')


Waiting for changeset to be created..
Waiting for stack create/update to complete
Successfully created/updated stack - aws-config-aggregator
It can take 10 minutes for the aggregator to be ready to field a query


## Test AWS Config Aggregator Query
The following query will display the first 100 the running EC2 instances in the organization. Remember, it may take a few minutes for the aggregator to completely deployed across the organization.

The **jupyter_awsconfig** is a helper library to simplify calling aws config queries and returning a pandas dataframe. 

In [75]:
import json
import jupyter_awsconfig

import importlib
importlib.reload(jupyter_awsconfig)

query_expression = f"""
SELECT
  accountId,
  awsRegion,
  resourceId,
  configuration.instanceType,
  availabilityZone,
  configuration.state.name
WHERE
  resourceType = 'AWS::EC2::Instance'
"""

df = jupyter_awsconfig.run_query(query_expression, 100)
display(df)

{'SelectFields': [{'Name': 'accountId'}, {'Name': 'awsRegion'}, {'Name': 'resourceId'}, {'Name': 'configuration.instanceType'}, {'Name': 'availabilityZone'}, {'Name': 'configuration.state.name'}]}


,accountId,awsRegion,resourceId,configuration.instanceType,availabilityZone,configuration.state.name
0,669399753706,us-east-1,i-0032d7951092d67cb,m5.large,us-east-1d,running
1,669399753706,ap-northeast-1,i-00b460faad1f58124,m4.large,ap-northeast-1a,stopped
2,669399753706,us-west-2,i-00ca087d43d41c6b8,m4.large,us-west-2a,running
3,669399753706,ap-southeast-2,i-01b1373cbf6032b91,t3.micro,ap-southeast-2c,stopped
4,669399753706,us-west-2,i-0245fb16c9a734f49,m4.large,us-west-2a,stopped
5,383086473915,us-east-1,i-0340922ec3bee66fc,t2.micro,us-east-1b,stopped
6,669399753706,eu-west-2,i-040ed0467f78f9001,m4.large,eu-west-2a,stopped
7,669399753706,ap-northeast-2,i-04b0f67316b4c0420,t2.micro,ap-northeast-2a,running
8,383086473915,us-east-1,i-051e3ad3391dfaada,c5.xlarge,us-east-1a,stopped
9,383086473915,us-east-1,i-055a29b981166fd51,c5.large,us-east-1c,running
